# Training on scale

### Create an environment

In [ ]:
%%writefile 'Environments/create-environment.yml'
$schema: https://azuremlschemas.azureedge.net/latest/environment.schema.json
name: ignite-transferlearning-pytorch
version: 1
image: mcr.microsoft.com/azureml/curated/pytorch-1.10-ubuntu18.04-py38-cuda11-gpu:36
description: Training enviroment PyTorch 1.10 / CUDA 11 / GPU

In [ ]:
!az config set defaults.workspace="Cybertron"
!az config set defaults.group="Cybertron-rg"     
!az ml environment create --file Environments/create-environment.yml

### Create train component

In [ ]:
%%writefile 'Components/train.yml'
$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
type: command

name: ignite_train_model
display_name: Ignite - Train model
version: 1

inputs:
  training_data:
    type: path
  epochs:
    type: integer
    default: 8
  learning_rate:
    type: number
    default: 0.001
  momentum: 
    type: number
    default: 0.9
  tag: 
    type: string
    default: "Ignite"

outputs:
  train_output:
    type: path

code: ./src

environment: azureml:ignite-transferlearning-pytorch:1

command: python train.py --data_path ${{inputs.training_data}} --model_output_path ${{outputs.train_output}} --num_epochs ${{inputs.epochs}} --learning_rate ${{inputs.learning_rate}} --momentum ${{inputs.momentum}} --tag ${{inputs.tag}} 

In [ ]:
!az ml component create --file Components/train.yml

In [ ]:
%%writefile 'Components/register.yml'
$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
type: command

name: ignite_register_model
display_name: Ignite - Register model after run
version: 3

inputs:
  model_assets_path:
    type: path
  model_name:
    type: string
    default: "simpsons-classification"
  model_file_name:
    type: string
    default: "model.pth"
  label_file_name:
    type: string
    default: "labels.txt" 

code: ./src

environment: azureml:ignite-transferlearning-pytorch:1

command: python register.py --model_assets_path ${{inputs.model_assets_path}} --model_name ${{inputs.model_name}} --model_file_name ${{inputs.model_file_name}} --label_file_name ${{inputs.label_file_name}}

In [ ]:
!az ml component create --file Components/register.yml

## Create the pipeline

In [ ]:
%%writefile 'Pipeline/trainings_pipeline.yml'
$schema: https://azuremlschemas.azureedge.net/latest/pipelineJob.schema.json
type: pipeline
experiment_name: Ignite-Pipeline
description: Pipeline to create a Lego Classifier in PyTorch
display_name: Ignite - Pipeline - Test

settings:
  default_datastore: azureml:workspaceartifactstore
  default_compute: azureml:UltraMagnus


inputs:
  training_data:
    mode: ro_mount
    path: azureml:LegoCharactersV2:2

jobs:
  train_model:
    type: command
    component: azureml:ignite_train_model:1
    inputs:
      training_data: ${{parent.inputs.training_data}}
      epochs: 16
      tag: "TEST7"
    outputs:
      train_output: 
        mode: upload
      
  register_pytorch:
    type: command
    component: azureml:ignite_register_model:3
    inputs:
      model_assets_path: ${{parent.jobs.train_model.outputs.train_output}}
      model_name: "pipeline-lego-pytorch"
      model_file_name: "model.pth"

In [ ]:
!az ml job create --file Pipeline/trainings_pipeline.yml

### Deploy the model

In [ ]:
%%writefile 'Deployment/deployment.yml'
$schema: https://azuremlschemas.azureedge.net/latest/managedOnlineDeployment.schema.json
endpoint_name: lego-endpoint-test
name: pipeline-lego-pytorch-1-v2
app_insights_enabled: true
model: azureml:pipeline-lego-pytorch:1
code_configuration:
  code: ./
  scoring_script: score.py
environment: azureml:AzureML-pytorch-1.7-ubuntu18.04-py37-cpu-inference:32
instance_type: Standard_F8s_v2
instance_count: 1

In [ ]:
!az ml online-deployment create --file Deployment/deployment.yml

In [ ]:
%%writefile 'test-images.txt'
https://raw.githubusercontent.com/hnky/dataset-lego-figures/master/_test/Krusty.jpg
https://raw.githubusercontent.com/hnky/dataset-lego-figures/master/_test/Flanders.jpg
https://raw.githubusercontent.com/hnky/dataset-lego-figures/master/_test/Homer.jpg
https://raw.githubusercontent.com/hnky/dataset-lego-figures/master/_test/Lisa.jpg
https://raw.githubusercontent.com/hnky/dataset-lego-figures/master/_test/MrBurns.jpg
https://raw.githubusercontent.com/hnky/dataset-lego-figures/master/_test/Wiggum.jpg
https://raw.githubusercontent.com/hnky/dataset-lego-figures/master/_test/wonderwomen.jpg
https://raw.githubusercontent.com/hnky/dataset-lego-figures/master/_test/Emperor-Palpatine.jpg
https://raw.githubusercontent.com/hnky/dataset-lego-figures/master/_test/Jar-Jar-Binks.jpg
https://raw.githubusercontent.com/hnky/dataset-lego-figures/master/_test/Kit-Fisto.jpg
https://raw.githubusercontent.com/hnky/dataset-lego-figures/master/_test/Yoda.jpg
https://raw.githubusercontent.com/hnky/dataset-lego-figures/master/_test/bat-mite.jpg
https://raw.githubusercontent.com/hnky/dataset-lego-figures/master/_test/batman.jpg
https://raw.githubusercontent.com/hnky/dataset-lego-figures/master/_test/cheetah.jpg
https://raw.githubusercontent.com/hnky/dataset-lego-figures/master/_test/stargirl.jpg
https://raw.githubusercontent.com/hnky/dataset-lego-figures/master/_test/wonderwomen.jpg

In [ ]:
# Handle to the workspace
from azure.ai.ml import MLClient
from azure.ai.ml import command, Input, Output
from azure.ai.ml.entities import Environment
from azure.ai.ml.entities import Data, Model
from azure.ai.ml.constants import AssetTypes

# Authentication package
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()

ml_client = MLClient(
    credential=credential,
    subscription_id = "431dbae5-40ca-438a-8daf-77d7d5580b41",
    resource_group_name = "Cybertron-RG",
    workspace_name = "Cybertron",
)

endpnt = ml_client.online_endpoints.get("lego-endpoint-test")




In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
import cv2
import urllib
import cv2
import numpy as np
import json
import urllib.request
import json


F = plt.figure(1, (20,20))
grid = AxesGrid(F, 111, nrows_ncols=(3, 4), axes_pad=0, label_mode="1")

with open('test-images.txt', 'rt') as lf:
    global testimages
    testimages = [l.strip() for l in lf.readlines()]
    
def url_to_image(url):
    with urllib.request.urlopen(url) as url:
        s = url.read()
    image = np.asarray(bytearray(s), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (352, 352))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image


i = 0
for img_name in testimages[0:12]:
    
    data = { "image":img_name }

    body = str.encode(json.dumps(data))
    url = 'https://lego-endpoint-test.westeurope.inference.ml.azure.com/score'
    api_key = 'yRPzrB3YLC9FkKaki1Mu6RTFkQUMVedW'

    headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key) }
    req = urllib.request.Request(url, body, headers)
    response = urllib.request.urlopen(req)
    JSON_object = json.loads(response.read())

    # Download image
    img = url_to_image(img_name)
 
    # Draw result on image
    cv2.rectangle(img, (0,260),(352,352),(255,255,255), -1)
    cv2.putText(img, "{} - {}%".format(JSON_object['prediction'],JSON_object['scores'][JSON_object['prediction']]),(10, 300), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0,0,0),2,cv2.LINE_AA)

    # Show image in grid
    grid[i].imshow(img)
    i = i+1